In [1]:
import pandas as pd 
import numpy as np 
import sys
sys.path.append('/home/alexabades/recsys')

In [2]:
dt = pd.read_csv("../data/processed/ml-1m/ml-1m.train.rating", sep="\t",names=["UserID", "MovieID", "Rating", "Timestamp"])

In [5]:
# Filter out rows where Rating is 0, then group by MovieID and count non-null Ratings
popular_items = list(dt[dt["Rating"] != 0].groupby("MovieID").count()["Rating"].sort_values(ascending=False).head(10).index)
popular_items

[104, 124, 44, 64, 113, 48, 97, 132, 22, 128]

In [24]:
# Initial data
gt_items = [1, 2, 3]
gt_scores = [5, 3, 7]

# Sorting gt_scores in descending order and keeping track of the original indices
sorted_indices = sorted(range(len(gt_scores)), key=lambda i: gt_scores[i], reverse=True)
sorted_gt_scores = [gt_scores[i] for i in sorted_indices]
sorted_gt_items = [gt_items[i] for i in sorted_indices]

sorted_gt_scores, sorted_gt_items


([7, 5, 3], [3, 1, 2], [3, 5, 7])

In [6]:
from typing import List
def getHitRatio(ranklist: List[int], gtItem: int) -> int:
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0
def getHitRatio1(ranklist: List[int], gtItem: int) -> int:
    if gtItem in ranklist:
      return 1
    return 0

def getHR(ranklist: List[int], gtItem: List[int]) -> int:
  for item in ranklist:
    if item in gtItem:
      return 1
  return 0

ranklist = [1, 2, 3]
gtItem = [1, 4]

ranklist.index(3)
# getHitRatio(ranklist, gtItem), getHitRatio1(ranklist, gtItem), getHR(ranklist,  gtItem)

2

In [10]:
type(0)

int

In [59]:
def getExponentialDCG(
    ranklist: List[int], scores: List[float], gtItem: int
) -> float | int:
    """
    Computes the Exponential Discounted Cumulative Gain (DCG) for a given item
    based on its position and relevance score.

    Parameters:
    - ranklist (List[int]): Ranked list of item IDs.
    - scores (List[float]): Relevance scores corresponding to items in ranklist.
    - gtItem (int): ID of the item to calculate DCG for.

    Returns:
    - float | int: Exponential DCG value if gtItem is in ranklist, 0 otherwise.
    """

    if gtItem not in ranklist:
        return 0
    idx = ranklist.index(gtItem)
    rel_idx = scores[idx]
    return (2**rel_idx) / (math.log2(idx + 2))

[False, False, True]

In [71]:
from typing import List
def getBinaryDCG(ranklist:List[int], gt_items:List[int]):
  dcg = 0
  for idx, item_id in enumerate(ranklist):
    if item_id in gt_items:
      dcg += (1) / (math.log2(idx + 2))
  return dcg
def getBinaryIDCG(gt_items:List[int]):
  idcg = 0
  for idx, item_id in enumerate(gt_items):
    idcg += (1) / (math.log2(idx + 2))
  return idcg
def getBinaryIDCG_optimized(gt_items: List[int]):
  # Precompute log values to avoid recalculating them for each element
  log_values = [math.log2(idx + 2) for idx in range(len(gt_items))]
  idcg = sum(1 / log_values[idx] for idx in range(len(gt_items)))
  return idcg

def getLinearDCG(ranklist:List[int], ranklist_scores:List[int|float], gt_items:List[int]) -> float:
  dcg = 0 
  for idx, rel in enumerate(ranklist_scores):
    dcg += (rel) / math.log2(idx + 2)
  return dcg

def getExponentialDCG(ranklist:List[int], ranklist_scores:List[int|float], gt_items:List[int]) -> float:
  dcg = 0 
  for idx, rel in enumerate(ranklist_scores):
    if ranklist[idx] in gt_items:
      dcg += (2**rel) / math.log2(idx + 2)
  return dcg





getBinaryDCG(ranklist, gt_items), getBinaryIDCG(ranklist)

(1.5, 2.1309297535714578)

In [72]:
def getIDCG1(ranklist):
  ranklist_sort = sorted(ranklist, reverse=True)
  idcg = 0
  for idx, rel in enumerate(ranklist_sort):
    idcg += (2**rel) / (math.log2(idx + 2))
  return idcg


def getIDCG(ranklist):
    # Precompute log values to avoid recalculating them for each element
    log_values = [math.log2(idx + 2) for idx in range(len(ranklist))]
    idcg = sum((2**rel) / log_values[idx] for idx, rel in enumerate(ranklist))
    return idcg

from time import time
ranklist = [random.randint(0,50)/10 for i in range(0,100)] 
start1 = time()
for i in range(1000):
  idcg = getBinaryIDCG(ranklist)
end1 = time()
start2 = time()
for i in range(1000):
  idcg = getBinaryIDCG_optimized(ranklist)
end2 = time()

print(f"Time 1: {end1-start1}")
print(f"Time 2: {end2-start2}")


Time 1: 0.030371665954589844
Time 2: 0.03425908088684082


In [ ]:
a =  [3, 1, 2]
map(a, a.index)

In [18]:
gt_items = [1, 2, 3]
gt_scores = [5, 3, 7]

sort, indx = sorted(gt_scores)

ValueError: too many values to unpack (expected 2)

In [15]:
import math
def DCG(ranklist, gtItem: int) -> int:
    if gtItem not in ranklist:
        return 0
    return ranklist.index(gtItem)

idx = DCG(popular_items, 104)

(1)/(math.log2(idx+2))

1.0

In [27]:
dt_test = pd.read_csv("../data/processed/ml-1m/ml-1m.test.rating", sep="\t",names=["UserID", "MovieID", "Rating", "Timestamp"])

In [38]:
dt_test.head()

,UserID,MovieID,Rating,Timestamp
0,0,25,5,978824351
1,1,133,3,978300174
2,2,207,4,978298504
3,3,208,4,978294282
4,4,222,2,978246585


In [34]:
popular_items

Index([104, 124, 44, 64, 113, 48, 97, 132, 22, 128], dtype='int64', name='MovieID')

In [56]:
hts = []
for row in dt_test.iterrows():
  if row[1]["MovieID"] in popular_items:
    hts.append(1)
  else:
    hts.append(0)
  

In [60]:
sum(hts)/len(hts)

0.022019867549668875

In [64]:
dt_test[["MovieID"]].map(lambda x : 1 if x in list(popular_items) else 0)["MovieID"].mean()

MovieID    0.02202
dtype: float64

In [65]:
# Calculate Hit Ratio using a more streamlined approach
average_ht = dt_test["MovieID"].apply(lambda x: 1 if x in popular_items else 0).mean()


In [66]:
average_ht

0.022019867549668875

In [67]:
def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

In [69]:
# Assuming dt_test is your test DataFrame and popular_items is a list of top-K item IDs
hts = []
for _, row in dt_test.iterrows():
    hts.append(getHitRatio(popular_items, row["MovieID"]))

In [87]:
import math 

def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0

In [88]:
# Assuming dt_test is your test DataFrame and popular_items is a list of top-K item IDs
ncdgs = []
for _, row in dt_test.iterrows():
    ncdgs.append(getNDCG(popular_items, row["MovieID"]))

In [90]:
ncdg = sum(ncdgs)/len(ncdgs)

0.010542016489001741

In [91]:
import math

def getDCG(ranklist, gtItem):
    """Calculate DCG based on the position of the ground truth item in the ranked list."""
    for i, item in enumerate(ranklist):
        if item == gtItem:
            return math.log(2) / math.log(i + 2)  # Using log base 2
    return 0

def getIDCG(K):
    """Calculate Ideal DCG for the top K items. Assuming binary relevance (1 for relevant, 0 for not relevant),
    the ideal case would have the relevant item at the top of the list."""
    # For binary relevance, the best case is having the item at the top, hence K=1
    if K > 0:
        return math.log(2) / math.log(2)  # First position relevance
    else:
        return 0

# Assume popular_items is sorted by popularity
ncdgs = []
K = 10  # Assuming we're evaluating top-K items for NDCG
idcg = getIDCG(K)  # Calculate IDCG once since it's constant for binary relevance with K=1

for _, row in dt_test.iterrows():
    dcg = getDCG(popular_items[:K], row["MovieID"])
    ncdg = dcg / idcg if idcg > 0 else 0
    ncdgs.append(ncdg)

average_ncdg = sum(ncdgs) / len(ncdgs)


In [93]:
math.log(2) / math.log(2)

1.0

In [80]:
filepath = "../data/processed/ml-1m/ml-1m"

In [94]:
import pandas as pd


class PopularItem:
    def __init__(self, path) -> None:
        self.path = path
        # Load data as soon as an instance is created
        self.dt_train, self.dt_test = self._load_data()

    def _load_data(self):
        # Private method to load data
        train_extension = ".train.rating"
        test_extension = ".test.rating"

        dt_train = pd.read_csv(
            f"{self.path}{train_extension}",
            sep="\t",
            names=["UserID", "MovieID", "Rating", "Timestamp"],
        )
        dt_test = pd.read_csv(
            f"{self.path}{test_extension}",
            sep="\t",
            names=["UserID", "MovieID", "Rating", "Timestamp"],
        )

        return dt_train, dt_test

    def calculate_hit_ratio(self, K):
        # Calculate popular items based on training data
        self.popular_items = list(
            self.dt_train[self.dt_train["Rating"] != 0]
            .groupby("MovieID")
            .count()["Rating"]
            .sort_values(ascending=False)
            .head(K)
            .index
        )
        # Calculate hit ratio based on test data
        average_ht = (
            self.dt_test["MovieID"]
            .apply(lambda x: 1 if x in self.popular_items else 0)
            .mean()
        )
        return average_ht

    def getDCG(ranklist, gtItem):
        """Calculate DCG based on the position of the ground truth item in the ranked list."""
        for i, item in enumerate(ranklist):
            if item == gtItem:
                return math.log(2) / math.log(i + 2)  # Using log base 2
        return 0
    
    def calculate_ndcg_ratio_binary(self,K):
        ncdgs = []
        idcg = 1
        for _, row in dt_test.iterrows():
            dcg = getDCG(popular_items[:K], row["MovieID"])
            ncdg = dcg / idcg  
            ncdgs.append(ncdg)
        average_ncdg = sum(ncdgs) / len(ncdgs)
        return average_ncdg

In [95]:
model = PopularItem(path=filepath)

In [96]:
model.calculate_hit_ratio(10)

0.022019867549668875

In [97]:
model.calculate_ndcg_ratio_binary(10)

0.010542016489001741

In [98]:
import os

# The path you're working with
path = 'src/config/nfc/ml-1m-1.yaml'

# Extract the base name (file name with extension)
base_name = os.path.basename(path)

# Split the base name by '.' and discard the extension
file_name_without_extension = os.path.splitext(base_name)[0]

print(file_name_without_extension)


ml-1m-1
